In [1]:
from tsai.all import *
import tensorflow as tf
from tensorflow import keras
import tempfile
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, confusion_matrix

In [2]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [3]:
path = r"./three_training.txt"
Training_timestep = pd.read_csv(path, sep = " ")
Training = pd.DataFrame(Training_timestep)
Training_copy = Training.copy()

In [4]:
training = Training_copy.iloc[:, 5:]

In [7]:
Training_copy

,ID,Feature,X,Y,Z,0.05,0.1,0.15,0.2,0.25,...,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1,Label
0,1.0,eet,-1.217015,9.605834,12.868542,0.009123,0.028492,0.048137,0.072138,0.111565,...,0.493622,0.551709,0.611433,0.660348,0.698846,0.733547,0.776131,0.827058,0.871621,1
1,1.0,pln,-1.217015,9.605834,12.868542,0.504139,0.718633,0.581406,0.353408,0.200452,...,0.086507,0.106281,0.146510,0.150176,0.162844,0.174045,0.198820,0.221131,0.248961,1
2,1.0,lnr,-1.217015,9.605834,12.868542,0.228755,0.087486,0.307852,0.526284,0.623806,...,0.502120,0.513251,0.514513,0.546258,0.554900,0.560645,0.541777,0.507077,0.465792,1
3,2.0,eet,5.614500,7.809000,12.703750,0.009634,0.027650,0.057777,0.092929,0.129222,...,0.405972,0.487673,0.544681,0.588438,0.636624,0.681728,0.715538,0.747381,0.786171,1
4,2.0,pln,5.614500,7.809000,12.703750,0.605983,0.749310,0.314200,0.221937,0.398596,...,0.262406,0.169332,0.111424,0.136674,0.214696,0.267766,0.350590,0.460112,0.486856,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959995,319999.0,pln,-28.999500,12.733000,13.964000,0.354523,0.306881,0.304572,0.262041,0.270267,...,0.105064,0.063420,0.066048,0.220701,0.244300,0.265468,0.278936,0.316067,0.332315,1
959996,319999.0,lnr,-28.999500,12.733000,13.964000,0.099792,0.297157,0.260421,0.406861,0.375552,...,0.577973,0.607442,0.575644,0.425919,0.391237,0.343409,0.263866,0.172647,0.130121,1
959997,320000.0,eet,-2.114515,7.881634,10.915292,0.010208,0.033108,0.056588,0.085469,0.118940,...,0.470754,0.520687,0.577135,0.637429,0.694187,0.739115,0.775346,0.808241,0.841436,1
959998,320000.0,pln,-2.114515,7.881634,10.915292,0.556721,0.279945,0.286952,0.235921,0.196030,...,0.367097,0.296408,0.196038,0.115270,0.204719,0.269311,0.311610,0.335434,0.340747,1


In [ ]:
window_length = 3
stride = None
start = 0
get_x = training.columns[:-1]
get_y = "Label"
horizon = 0
seq_first = True
X, Y = SlidingWindow(window_length, stride = stride, start=start, get_x =get_x, get_y =get_y, horizon = horizon, seq_first = seq_first)(training)
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, stratify =Y, random_state=45)

In [ ]:
x = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [ ]:
scalers = {}
for i in range(x.shape[1]):
    scalers[i] = StandardScaler()
    x[:, i, :] = scalers[i].fit_transform(x[:, i, :])

In [ ]:
seed = 8
np.random.seed(seed)

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []

In [ ]:
METRICS = [
    keras.metrics.TruePositives(name = "tp"),
    keras.metrics.FalsePositives(name = "fp"),
    keras.metrics.TrueNegatives(name = "tn"),
    keras.metrics.FalseNegatives(name = "fn"),
    keras.metrics.BinaryAccuracy(name = "accuracy"),
    keras.metrics.Precision(name = "precision"),
    keras.metrics.Recall(name = "recall"),
    keras.metrics.AUC(name="auc"),
    keras.metrics.AUC(name="prc", curve="PR")
]

In [ ]:
def showResults(test, pred):
    print(classification_report(test, pred))
    accuracy = accuracy_score(test, pred)
    precision=precision_score(test, pred, average='weighted',labels=np.unique(pred))
    f1Score=f1_score(test, pred, average='weighted',labels=np.unique(pred)) 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(test, pred)
    print(cm)

In [ ]:
# LSTM-FCN
for train, valid in kfold.split(x, y):
    def generate_lstmfcn(input_shape):
        input_layer = keras.layers.Input(input_shape)

        x = keras.layers.LSTM(8)(input_layer)
        x = keras.layers.Dropout(0.8)(x)

        y = keras.layers.Permute((2, 1))(input_layer)
        y = keras.layers.Conv1D(128, 8, padding='same', kernel_initializer='he_uniform',kernel_regularizer= keras.regularizers.l2(0.001))(y)
        y = keras.layers.BatchNormalization()(y)
        y = keras.layers.Activation('relu')(y)

        y = keras.layers.Conv1D(256, 5, padding='same', kernel_initializer='he_uniform',kernel_regularizer= keras.regularizers.l2(0.001))(y)
        y = keras.layers.BatchNormalization()(y)
        y = keras.layers.Activation('relu')(y)

        y = keras.layers.Conv1D(128, 3, padding='same', kernel_initializer='he_uniform',kernel_regularizer= keras.regularizers.l2(0.001))(y)
        y = keras.layers.BatchNormalization()(y)
        y = keras.layers.Activation('relu')(y)

        y = keras.layers.GlobalAveragePooling1D()(y)

        x = keras.layers.concatenate([x, y])

        output_layer = keras.layers.Dense(1, activation='sigmoid')(x)

        return keras.models.Model(inputs=input_layer, outputs=output_layer)

    model =  generate_lstmfcn(input_shape=X_train.shape[1:]) 
    
    EPOCHS = 100
    BATCH_SIZE = 256
    
    callbacks = [
        keras.callbacks.ReduceLROnPlateau(
           monitor="val_loss", factor=0.5, patience=10, min_lr=0.0001
        ),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1),
    ]
    
    model.compile(
         optimizer="Adam",
         loss="binary_crossentropy",
         metrics=METRICS,)
    
    model_history = model.fit(
       x[train],
       y[train],
       batch_size=BATCH_SIZE,
       epochs=EPOCHS,
       validation_data = (x[valid],y[valid]),
       callbacks=callbacks,
       verbose=1,)
    
    yclasses = (model.predict(x[valid]) > 0.5).astype("int32")
    showResults(y[valid], yclasses)

In [ ]:
# Residual networks
for train, valid in kfold.split(x, y):
    def build_model(input_shape):
        n_feature_maps=64
        input_layer = keras.layers.Input(input_shape)

        conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same',kernel_regularizer= keras.regularizers.l2(0.001))(input_layer)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same',kernel_regularizer= keras.regularizers.l2(0.001))(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same',kernel_regularizer= keras.regularizers.l2(0.001))(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same',kernel_regularizer= keras.regularizers.l2(0.001))(input_layer)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_1 = keras.layers.add([shortcut_y, conv_z])
        output_block_1 = keras.layers.Activation('relu')(output_block_1)

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same',kernel_regularizer= keras.regularizers.l2(0.001))(output_block_1)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same',kernel_regularizer= keras.regularizers.l2(0.001))(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same',kernel_regularizer= keras.regularizers.l2(0.001))(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same',kernel_regularizer= keras.regularizers.l2(0.001))(output_block_1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_2 = keras.layers.add([shortcut_y, conv_z])
        output_block_2 = keras.layers.Activation('relu')(output_block_2)

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same',kernel_regularizer= keras.regularizers.l2(0.001))(output_block_2)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same',kernel_regularizer= keras.regularizers.l2(0.001))(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same',kernel_regularizer= keras.regularizers.l2(0.001))(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

            # no need to expand channels because they are equal
        shortcut_y = keras.layers.BatchNormalization()(output_block_2)

        output_block_3 = keras.layers.add([shortcut_y, conv_z])
        output_block_3 = keras.layers.Activation('relu')(output_block_3)

        gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

        output_layer = keras.layers.Dense(1, activation='sigmoid')(gap_layer)

        return keras.models.Model(inputs=input_layer, outputs=output_layer)

    model = build_model(input_shape=X.shape[1:])
    
    EPOCHS = 100
    BATCH_SIZE = 256
    
    callbacks = [
        keras.callbacks.ReduceLROnPlateau(
           monitor="val_loss", factor=0.5, patience=10, min_lr=0.0001
        ),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1),
    ]
    
    model.compile(
         optimizer="Adam",
         loss="binary_crossentropy",
         metrics=METRICS,)
    
    model_history = model.fit(
       x[train],
       y[train],
       batch_size=BATCH_SIZE,
       epochs=EPOCHS,
       validation_data = (x[valid],y[valid]),
       callbacks=callbacks,
       verbose=1,)
    
    yclasses = (model.predict(x[valid]) > 0.5).astype("int32")
    showResults(y[valid], yclasses)

In [ ]:
nb_epoch =len(model_history.history['loss'])
learning_rate = model_history.history['lr']
xc = range(nb_epoch)
plt.figure(3, figsize= (7,5))
plt.plot(xc, learning_rate)
plt.xlabel('num of Epochs',fontsize = 16)
plt.ylabel('learning rate',fontsize = 16)
plt.title('Learning rate',fontsize = 20)
plt.grid(True)
plt.style.use('seaborn-ticks')

In [ ]:
scores = model.evaluate(x[train], y[train], verbose=0)
scores

In [ ]:
model.save("./total_model")

In [ ]:
path = r"./testing.txt"
Testing_timestep = pd.read_csv(path, sep = " ")
Testing = pd.DataFrame(Testing_timestep)
Testing_copy = Testing.copy()

In [ ]:
testing = Testing_copy.iloc[:, 5:]

In [ ]:
window_length =3
stride = None
start = 0
get_x = testing.columns[:-1]
get_y = "Label"
horizon = 0
seq_first = True
X_test, Y_test = SlidingWindow(window_length, stride = stride, start=start, get_x =get_x, get_y =get_y, horizon = horizon, seq_first = seq_first)(testing)
X_test.shape

In [ ]:
scalers = {}
for i in range(X_test.shape[1]):
    scalers[i] = StandardScaler()
    X_test[:, i, :] = scalers[i].fit_transform(X_test[:, i, :])

In [ ]:
train_prediction = model.predict(x[train], batch_size =BATCH_SIZE)
test_prediction = model.predict(X_test, batch_size=BATCH_SIZE)

In [ ]:
test_prediction.shape

In [ ]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

In [ ]:
model_result = model.evaluate(X_test, Y_test, batch_size= BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, model_result):
    print(name, ': ', value)
print()

plot_cm(Y_test, test_prediction)

In [ ]:
def plot_prc(name, labels, predictions, **kwargs):
    precision, recall, _ = sklearn.metrics.precision_recall_curve(labels, predictions)

    plt.plot(precision, recall, label=name, linewidth=2, **kwargs)
    plt.xlabel('Recall',fontsize = 18)
    plt.ylabel('Precision', fontsize =18)
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

In [ ]:
plot_prc("Train", y[train] , train_prediction, color=colors[0])
plot_prc("Test",Y_test, test_prediction, color=colors[0], linestyle='--')
plt.legend(loc='lower left')

In [ ]:
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 16

plt.rc('font',size = BIGGER_SIZE)
plt.rc('axes',titlesize = BIGGER_SIZE)
plt.rc('axes', labelsize = MEDIUM_SIZE)
plt.rc('xtick', labelsize = BIGGER_SIZE)
plt.rc('ytick', labelsize =BIGGER_SIZE)
plt.rc('legend', fontsize = BIGGER_SIZE)
plt.rc('figure', titlesize = BIGGER_SIZE)


In [ ]:
def plot_metrics(history):
  metrics = ['accuracy']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch', fontsize = 18)
    plt.ylabel(name, fontsize = 18)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend(loc='lower right')

In [ ]:
plot_metrics(model_history)

In [ ]:
def plot_roc(name, labels, predictions, **kwargs):
  fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives [%]', fontsize = 18)
  plt.ylabel('True positives [%]', fontsize = 18)
  plt.xlim([-0.5,100])
  plt.ylim([0,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

In [ ]:
plot_roc("Train", y[train] , train_prediction, color=colors[0])
plot_roc("Test", Y_test, test_prediction, color=colors[0], linestyle='--')
plt.legend(loc='lower right')

In [ ]:
path = r"./testing.txt"
S1_timestep = pd.read_csv(path, sep = " ")
S1 = pd.DataFrame(S1_timestep)
S1_copy = S1.copy()

In [ ]:
s1 = S1_copy.iloc[:, 5:]

In [ ]:
window_length =3
stride = None
start = 0
get_x = s1.columns[:-1]
get_y = "Label"
horizon = 0
seq_first = True
X_S1, Y_S1 = SlidingWindow(window_length, stride = stride, start=start, get_x =get_x, get_y =get_y, horizon = horizon, seq_first = seq_first)(s1)
X_S1.shape

In [ ]:
scalers = {}
for i in range(X_S1.shape[1]):
    scalers[i] = StandardScaler()
    X_S1[:, i, :] = scalers[i].fit_transform(X_S1[:, i, :])

In [ ]:
s1_predicted = (model.predict(X_S1) > 0.5).astype("int32")

In [ ]:
np.savetxt("./S6total_label.txt",s1_predicted)

In [ ]:
len(s1_predicted)

In [ ]:
new_input_layer = model.inputs
    # output is both the original as well as the before last layer
new_output_layer = [model.layers[-3].output, model.layers[-1].output]

new_feed_forward = keras.backend.function(new_input_layer, new_output_layer)

classes = np.unique(Y_test)

In [ ]:
cas = np.zeros(dtype=np.float, shape=(conv_out.shape[1]))

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
for c in classes:
        plt.figure()
        count = 0
        c_x_test = X_test[np.where(Y_test == 0)]
        for ts in c_x_test:
            ts = ts.reshape(1,-1,3)
            [conv_out, predicted] = new_feed_forward([ts])
            pred_label = np.argmax(predicted)
            orig_label = np.argmax(enc.transform([[c]]))
            if pred_label == 0:
                cas = np.zeros(dtype=np.float, shape=(conv_out.shape[1]))
                for k, w in enumerate(w_k_c[:, orig_label]):
                    cas += w * conv_out[0, :, k]

                minimum = np.min(cas)

                cas = cas - minimum

                cas = cas / np.max(cas)
                cas = cas * 100

                x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
                # linear interpolation to smooth
                f = interp1d(range(ts.shape[1]), ts[0, :, 0])
                y = f(x)
                f = interp1d(range(ts.shape[1]), cas)
                cas = f(x).astype(int)
                plt.scatter(x=x, y=y, c=cas, cmap='jet', marker='.', s=2, vmin=0, vmax=100, linewidths=0.0)
              
                count += 1
        cbar = plt.colorbar()
        plt.savefig("./fcn" + "-cam-" + "11" + "-class-" + str(int((c)))+ ".png", bbox_inches = 'tight', dpi =600)